In [1]:
# Transformer 모델 구축 - Transformer 질의응답(QA) 모델
# 학습 목표 - 실무에서 사용되는 파이프라인 이해 및 적용

In [2]:
# QA Pre-trained 모델 테스트
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

base_model = 'monologg/koelectra-base-v3-finetuned-korquad' # 한국어 KorQuAD 데이터셋으로 파인 튜닝된 KoELECTRA QA 모델
tokenizer = AutoTokenizer.from_pretrained(base_model) # 토크나이저 로드
model = AutoModelForQuestionAnswering.from_pretrained(base_model) # QA 태스크용 헤드가 포함된 모델 로드

question = '서울은 어디에 있나요?'
context = '서울은 대한민국의 수도이며, 한반도의 중서부에 위치해 있습니다.'

# QA 모델 구조: BERT 기반 QS 모델은 입력을 [CLS]질문[SEP]문맥[SEP] 형태로 받는다
# 질문과 문맥을 하나의 입력으로 합쳐서 토큰화 결과를 모델에 전달해야 답변을 얻을 수 있는 구조
# [CLS] 서울 은 어디 에 있 나요 ? [SEP] 서울 은 대한민국 의 수도 이며 , 한반도 의 중서부 에 위치 해 있습니다 . [SEP]
inputs = tokenizer(question, context, return_tensors='pt') # (batch_size,seq_len) (1,seq_len)

# 모델에 입력을 전달하여 start_logits, end_logits 출력
# - start_logits: 답변 시작 위치에 대한 확률 분포
# - end_logits: 답변 끝 위치에 대한 확률 분포
outputs = model(**inputs)

# torch.argmax() 가장 확률이 높은 인덱스를 선택
# - answer_start: 답변 시작 토큰 위치, answer_end: 답변 끝 토큰 위치
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1 # 마지막 인덱스 포함

# 토큰을 문자열로 변환
# - tokenizer.convert_ids_to_tokens(): 토큰 ID->토큰 문자열
# - tokenizer.convert_tokens_to_string(): 토큰 문자열->사람이 읽을 수 있는 문장
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end])
)

# Debug 확인
print('답변 시작 인덱스:', answer_start.item())
print('답변 끝 인덱스:', answer_end.item())

tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]) # 전체 토큰 확인
print('전체 토큰 시퀀스:', tokens)

answer_tokens = tokens[answer_start:answer_end] # 답변 토큰 범위 확인 
print('답변 토큰:', answer_tokens)

answer = tokenizer.convert_tokens_to_string(answer_tokens) # 최종 답변
print('최종 답변:', answer)

답변 시작 인덱스: 18
답변 끝 인덱스: 22
전체 토큰 시퀀스: ['[CLS]', '서울', '##은', '어디', '##에', '있', '##나', '##요', '?', '[SEP]', '서울', '##은', '대한민국', '##의', '수도', '##이', '##며', ',', '한반도', '##의', '중서', '##부', '##에', '위치', '##해', '있', '##습', '##니다', '.', '[SEP]']
답변 토큰: ['한반도', '##의', '중서', '##부']
최종 답변: 한반도의 중서부


In [3]:
# 데이터셋 로드 및 전처리
from datasets import load_dataset
from transformers import AutoTokenizer

# 한국어 KorQuAD 데이터셋: train(60,407), validation(5,774)
# - 데이터 구조: context(문서 본문), question(질문), answers(정답 스팬, 텍스트 + 시작 위치)
dataset = load_dataset('squad_kor_v1')

# validation 데이터셋을 반으로 나누어 test 데이터셋 생성
split_dataset = dataset['validation'].train_test_split(test_size=0.5, seed=42)
dataset['validation'] = split_dataset['train']
dataset['test'] = split_dataset['test']
print(dataset['train'].shape, dataset['validation'].shape, dataset['test'].shape)
print(dataset['train'][0])

(60407, 5) (2887, 5) (2887, 5)
{'id': '6566495-0-0', 'title': '파우스트_서곡', 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.', 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?', 'answers': {'text': ['교향곡'], 'answer_start': [54]}}


In [ ]:
# 토크나이저
# - KorQuAD 1.0/2.0: 위키 문서 기반, 질문-답변 쌍 포함, 내부적으로 ElectraTokenizerFast
# - Fast 토크나이저라서 offset_mapping 사용 가능
model_name = 'monologg/koelectra-base-v3-finetuned-korquad'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 전처리 함수: 질문 + 문맥 토큰화 + 정답 스팬 매핑
def preprocess_batched(examples):
    # 질문 + 문맥 토큰화 (offsets 위치 정보 포함)
    tokenized = tokenizer(
        examples['question'],
        examples['context'],
        truncation=True,
        padding='max_length',
        max_length=512,

        # return_offsets_mapping=True 시에는 offset_mapping 생성된다
        # 'offset_mapping': [
        #     (0, 0),   # [CLS]
        #     (0, 2),   # '베토'
        #     (2, 4),   # '벤'
        #     ...
        # ]
        return_offsets_mapping=True # offsets 위치 정보(offset_mapping 구조 생성)
    )

    start_positions = []
    end_positions = []

    # 예시 'answers': {'text': ['교향곡'], 'answer_start': [54]}
    for i, offsets in enumerate(tokenized['offset_mapping']):
        answer = examples['answers'][i] # 'answers': {'text': ['교향곡'], 'answer_start': [54]}
        start_char = answer['answer_start'][0] # 정답 시작 위치, 54
        end_char = start_char + len(answer['text'][0]) # 정답 시작 위치 + 텍스트 길이를 이용해 정답의 문자 단위 범위를 구한다, 54+3(교향곡)=57

        # tokenized.sequence_ids() 각 토큰이 질문(0)/문맥(1)/special token(None) 어디에 속하는지 알려준다
        # - 정답이 문맥(context)에 있으므로 sequence_ids[idx] == 1 인 토큰만 확인한다
        sequence_ids = tokenized.sequence_ids(i)

        start_pos = 0
        end_pos = 0

        # offset_mapping 순회
        for idx, (start, end) in enumerate(offsets):            
            if sequence_ids[idx] != 1: # context 토큰(1)만 사용, 질문/특수 토큰은 건너뜀
                continue
            if start <= start_char < end: # (시작,종료)의 정답 시작 문자가 포함된 토큰
                start_pos = idx # 정답 시작 문자가 포함된 index 번호 추가
            if start < end_char <= end: # (시작,종료)의 정답 종료 문자가 포함된 토큰
                end_pos = idx # 정답 종료 문자가 포함된 index 번호 추가
                break
        
        start_positions.append(start_pos) # 정답 시작 토큰 인덱스 적재
        end_positions.append(end_pos) # 정답 종료 토큰 인덱스 적재
    
    # 결과를 추가
    tokenized['start_positions'] = start_positions
    tokenized['end_positions'] = end_positions

    # offset_mapping은 학습에 필요 없으므로 제거한다, 메모리 낭비 방지
    tokenized.pop('offset_mapping')

    return tokenized

# 데이터셋 적용
tokenized_dataset = dataset.map(
    preprocess_batched, 
    batched=True, 
    remove_columns=dataset['train'].column_names
)

# 결과 확인
print(tokenized_dataset['train'][0].keys())
print('start_positions', tokenized_dataset['train'][0]['start_positions'])
print('end_positions', tokenized_dataset['train'][0]['end_positions'])

Map:   0%|          | 0/2887 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])
start_positions 46
end_positions 46


In [5]:
# collate_fn 정의 및 DataLoader 생성
from torch.utils.data import DataLoader

# collate_fn 정의: 텐서 변환
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(x['input_ids']) for x in batch])
    attention_mask = torch.stack([torch.tensor(x['attention_mask']) for x in batch])
    token_type_ids = torch.stack([torch.tensor(x['token_type_ids']) for x in batch])
    start_positions = torch.stack([torch.tensor(x['start_positions']) for x in batch])
    end_positions = torch.stack([torch.tensor(x['end_positions']) for x in batch])
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'token_type_ids': token_type_ids,
        'start_positions': start_positions,
        'end_positions': end_positions
    }

# DataLoader 구성: train, validation, test
train_loader = DataLoader(
    tokenized_dataset['train'], 
    batch_size=16, 
    shuffle=True,
    collate_fn=collate_fn
)
valid_loader = DataLoader(
    tokenized_dataset['validation'], 
    batch_size=16,
    shuffle=False,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    tokenized_dataset['test'], 
    batch_size=16,
    shuffle=False,
    collate_fn=collate_fn
)

In [18]:
# 모델 정의
# - Feature Extraction + LoRA Fine-tuning 조합
from transformers import AutoModelForQuestionAnswering
from peft import LoraConfig, get_peft_model
from torch.amp import autocast, GradScaler
import os

# GPU 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'PyTorch Version: {torch.__version__}, Device: {device}')

# AutoModelForQuestionAnswering 베이스 모델
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Feature Extraction: 본체 동결
for name, param in model.named_parameters():
    if 'lora' not in name: # LoRA 모듈이 아닌 경우
        param.requires_grad = False # 모델 본체 동결

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['query', 'value'], # attention 모듈에 LoRA 적용
    lora_dropout=0.1,
    bias='none',
    task_type='QUESTION_ANS'
)
model = get_peft_model(model, lora_config)
model.to(device) # 모델 -> gpu/cpu 이동

# 최적화 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
scaler = GradScaler()
num_epochs = 3 # 반복횟수 3회

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.0, path='./llm_models/20_transformer_qa/best_model.pt'):
        self.patience=patience
        self.min_delta=min_delta
        self.best_loss=None
        self.counter=0
        self.early_stop=False
        self.path=path
    
    def __call__(self, valid_loss, model):
        # 최조 손실값에 해당하는 모델 가중치 저장
        if self.best_loss is None:
            self.best_loss=valid_loss
            self.save_checkpoint(model)
        
        # 성능 개선 -> 최적 모델 갱신
        elif valid_loss < self.best_loss - self.min_delta:
            self.best_loss=valid_loss
            self.counter=0
            self.save_checkpoint(model)
        
        # 개선 없음
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop=True
    
    def save_checkpoint(self, model):
        # 디렉토리만 생성
        folder = os.path.dirname(self.path)
        if folder !='' and not os.path.exists(folder):
            os.makedirs(folder)
        # 모델 가중치 저장
        torch.save(model.state_dict(), self.path)
        print(f' Best model saved at {self.path}')

# Early Stopping 객체 생성
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

# 모델 확인
print(model)

PyTorch Version: 2.8.0+cu129, Device: cuda
PeftModelForQuestionAnswering(
  (base_model): LoraModel(
    (model): ElectraForQuestionAnswering(
      (electra): ElectraModel(
        (embeddings): ElectraEmbeddings(
          (word_embeddings): Embedding(35000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): ElectraEncoder(
          (layer): ModuleList(
            (0-11): 12 x ElectraLayer(
              (attention): ElectraAttention(
                (self): ElectraSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                   